In [ ]:
import configparser
import os
from trino.auth import BasicAuthentication
from trino.dbapi import connect

In [ ]:
config = configparser.ConfigParser()
config.read('config.ini')
creds = configparser.ConfigParser()
creds.read(os.getenv('HOME') + '/.ssh/trino.ini')

In [ ]:
print(f"{config.get('default','scheme')}://{config.get('default','host')}:{config.get('default','port')}/")
conn = connect(
    http_scheme=config.get('default', 'scheme'),
    host=config.get('default','host'),
    port=config.get('default','port'),
    user=creds.get('default', 'user'),
    auth=BasicAuthentication(creds.get('default','user'), creds.get('default','pass'))
)

In [ ]:
aggregatePercentiles = f"""

WITH
    stop  AS ( SELECT TRY(CAST(substr(stop_id,1,7) as INTEGER)) as bpuic, stop_lat, stop_lon FROM iceberg.com490_ice.sbb_stops_parquet_part WHERE year=2024 AND month=9 AND day=9),
    shape AS ( SELECT ST_GeomFromBinary(wkb_geometry) as geometry, name FROM iceberg.com490_ice.geo_parquet WHERE level='city' ),
    geo_tagged_stop AS ( SELECT stop.bpuic, stop.stop_lat, stop.stop_lon, shape.name FROM stop JOIN shape ON ST_Contains(shape.geometry, ST_Point(stop.stop_lon, stop.stop_lat))),
    geo_tagged_istdaten AS (
        SELECT day_of_week(istdaten.arr_actual) as day_week, hour(istdaten.arr_actual) hour_day, (istdaten.arr_actual - istdaten.arr_time) as arr_delay, (istdaten.dep_actual - istdaten.dep_time) as dep_delay, geo_tagged_stop.name
        FROM iceberg.com490_ice.sbb_istdaten_parquet_part AS istdaten JOIN geo_tagged_stop USING (bpuic)
    )
-- SELECT COUNT(), name FROM geo_tagged_istdaten GROUP BY name
SELECT AVG(arr_delay) as arr_delay, AVG(dep_delay) as dep_delay, COUNT() as num, approx_percentile(to_milliseconds(arr_delay)/1000, ARRAY[0.25,0.5,0.75]), hour_day, name FROM geo_tagged_istdaten WHERE day_week>=1 AND day_week <= 5  GROUP BY name,hour_day ORDER BY name,hour_day
"""

In [ ]:
from contextlib import closing
with closing(conn.cursor()) as cur:
    cur.execute(aggregatePercentiles)
    rows = cur.fetchall()
    for row in rows:
       print(f">>>>> {row}")

In [ ]:
conn.close()